In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor ,AdaBoostRegressor
import itertools as it
import time as time
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
#from pyearth import Earth

In [2]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [10]:
model_gb = GradientBoostingRegressor(n_estimators = 1300, loss = 'huber', max_depth = 9, 
                                     learning_rate = 0.08, subsample = 0.85, 
                                     random_state = 1).fit(X_train, y_train)
model_xgb = XGBRegressor(n_estimators = 4050, max_depth = 7, 
                         learning_rate = 0.01, subsample = 0.5, 
                         reg_lambda = 10, gamma = 0, colsample_bytree = 0.6,
                         random_state = 1).fit(X_train, y_train)   
model_ada = AdaBoostRegressor(estimator = DecisionTreeRegressor(max_depth=13),
                          n_estimators=1500, 
                          learning_rate=1.5, 
                          random_state=45).fit(X_train,y_train)                                  
model_bag = BaggingRegressor(random_state = 1, 
                             n_jobs = -1,
                             estimator = DecisionTreeRegressor(random_state = 1),
                             n_estimators = 1100, 
                             max_samples = 0.75, 
                             max_features = 1.0,
                             bootstrap_features = True, 
                             bootstrap = False).fit(X_train, y_train)
model_rf = RandomForestRegressor(n_estimators= 3000, 
                                 max_features = 7, 
                                 max_samples = 5000, 
                                 max_depth = 50, 
                                 random_state = 1, 
                                 n_jobs = -1).fit(X_train, y_train)

# models with greater than 0.7 RMSE
model_cat = CatBoostRegressor(verbose = False).fit(X_train, y_train)
model_lgbm = LGBMRegressor().fit(X_train, y_train)
#model_mars = Earth(max_terms = 600, max_degree = 6).fit(X_train, y_train)
# 0.8 RMSE
model_dt = DecisionTreeRegressor(max_leaf_nodes = 52,
                                 max_depth = 6, 
                                 min_samples_leaf = 8, 
                                 random_state = 45).fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
<ipython-input-10-59e842ef0d83>:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  n_jobs = -1).fit(X_t

In [7]:
#Voting ensemble: Averaging the predictions of all models
en = VotingRegressor(estimators = [('xgb', model_xgb), ('ada', model_ada),
                                   ('rf', model_rf),
                                   ('gb', model_gb), ('bag', model_bag)])
en.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.6,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.0...
                                                   max_samples=5000,
                                                   n_estimators=3000, n_jobs=-1,
                                                   random_state=1)),
                            ('gb',
                             GradientBoostingRegressor(learning_rate=0.08,
                                                       loss='huber',
                                                       max_depth=9,
                                                       n_estimators=1300,
                                                       random_state=1,
                                                       subsample=0.85)),
                            ('bag',
                             BaggingRegressor(bootstrap=False,
                                              bootstrap_features=True,
                                              estimator=DecisionTreeRegressor(random_state=1),
                                              max_samples=0.75,
                                              n_estimators=1100, n_jobs=-1,
                                              random_state=1))])

In [8]:
print("Ensemble model RMSE = ", np.sqrt(mean_squared_error(np.around(en.predict(X_test)), y_test)))

Ensemble model RMSE =  0.6510346794496848


In [11]:
en2 = VotingRegressor(estimators = [('xgb', model_xgb), ('ada', model_ada),
                                    ('rf', model_rf), ('cat', model_cat),
                                    ('lgbm', model_lgbm), ('gb', model_gb),
                                    ('bag', model_bag)])
en2.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.6,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.0...
                             <catboost.core.CatBoostRegressor object at 0x7f4bc52c2e90>),
                            ('lgbm', LGBMRegressor()),
                            ('gb',
                             GradientBoostingRegressor(learning_rate=0.08,
                                                       loss='huber',
                                                       max_depth=9,
                                                       n_estimators=1300,
                                                       random_state=1,
                                                       subsample=0.85)),
                            ('bag',
                             BaggingRegressor(bootstrap=False,
                                              bootstrap_features=True,
                                              estimator=DecisionTreeRegressor(random_state=1),
                                              max_samples=0.75,
                                              n_estimators=1100, n_jobs=-1,
                                              random_state=1))])

In [12]:
print("Ensemble model RMSE = ", np.sqrt(mean_squared_error(np.around(en.predict(X_test)), y_test)))

Ensemble model RMSE =  0.6510346794496848
